In [69]:
import gensim
import nltk
import pandas as pd
import numpy as np
from pandas import DataFrame
import spacy
from spacy.lang.en import English
nlp = English()
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-to-date!
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Package names is already up-to-date!
[nltk_data]    | Do

True

In [70]:
#Methods required for running the project on colab
from google.colab import drive
drive.mount("/content/drive/")


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [71]:
cd drive/My\ Drive/asap-aes(1)

[Errno 2] No such file or directory: 'drive/My Drive/asap-aes(1)'
/content/drive/My Drive/asap-aes(1)


In [0]:
#Utility function for reading Stanford's Glove vectors using gensim
def readWordVector(tmp_file):
    wv=KeyedVectors.load_word2vec_format(tmp_file, binary=False)
    return wv

In [74]:
file="glove.6B.300d.txt"
tmp_file = get_tmpfile('temp_word2vec.txt')
glove2word2vec(file, tmp_file)
wv = readWordVector(tmp_file)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
data = pd.read_csv('training_set_rel3.tsv', encoding = "ISO-8859-1",sep='\t')

In [78]:
data.head(6)

,essay_id,essay_set,essay,rater1_domain1,rater2_domain1,rater3_domain1,domain1_score,rater1_domain2,rater2_domain2,domain2_score,rater1_trait1,rater1_trait2,rater1_trait3,rater1_trait4,rater1_trait5,rater1_trait6,rater2_trait1,rater2_trait2,rater2_trait3,rater2_trait4,rater2_trait5,rater2_trait6,rater3_trait1,rater3_trait2,rater3_trait3,rater3_trait4,rater3_trait5,rater3_trait6
0,1,1,"Dear local newspaper, I think effects computer...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,"Dear @CAPS1 @CAPS2, I believe that using compu...",5,4,NaN,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,"Dear, @CAPS1 @CAPS2 @CAPS3 More and more peopl...",4,3,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1,"Dear Local Newspaper, @CAPS1 I have found that...",5,5,NaN,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1,"Dear @LOCATION1, I know having computers has a...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,1,"Dear @LOCATION1, I think that computers have a...",4,4,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Removing unnecessary columns since only domain1 score is required
y=data['domain1_score']
data=data.dropna(axis=1)
data=data.drop(axis=1,columns=['rater1_domain1','rater2_domain1'])

In [0]:
#As picked up from essay_set_description.xlsx
maximum_domain1_score=np.array([2,1,0,0,0,0,0,0])
minimum_domain1_score=np.array([12,6,3,3,4,4,30,60])

In [0]:
#Using Pythons Natural language library NLTK for getting the stopwords
stopwords = nltk.corpus.stopwords.words('english')
stopwords = [w.lower() for w in stopwords]
tagged_labels = ["PERSON","ORGANIZATION","LOCATION","DATE","TIME","MONEY","PERCENT","CAPS","NUM"]

In [0]:
from nltk import sent_tokenize, word_tokenize
import re
import string
#Split one essay to multiple sentences
#1D essay -> 2D Sentences
def split_sentence(essay):
  sentences = sent_tokenize(essay)
  return sentences
  

In [0]:
#Utility method to check if the word is tagged using NER in the dataset
def check_tagged_label(word):
    for w in tagged_labels:
        if w in word:
            return True
    return False

In [0]:
#Converting essay into words using a tokenizer trained on the training set
def essay_to_words(essay):
    sentence=[]
    words =tokenizer(essay)
    for word in words :
        if not word.lower() in stopwords and not check_tagged_label(word):
            sentence.append(word.lower())
    return sentence
    

In [0]:
#Converting all the essays into a list of words used in training
def create_sentence_vocab(essays):
    result = []
    for essay in essays:
        sentence = essay_to_words(essay)
        result.append(sentence)
    return result

In [0]:
#Converting the tokenized data to word index data so that it can be used to train our neural network
def create_essay_to_word_index(essays):
    output=[]
    maxlen=0
    for sequences in essays: 
        sequence = tokenizer(sequences)
        indices=[]
        for word in sequence:
            if not word.lower() in stopwords and not check_tagged_label(word):
              if word.lower() in vectorizer.vocabulary_:
                indices.append(vectorizer.vocabulary_[word.lower()])
              else:
                indices.append(vectorizer.vocabulary_["OOV"])
        maxlen=max(maxlen,len(indices))
        output.append(indices)
    return output,maxlen

In [0]:
#-----------------------------------TF-IDF Model------------------------------------------------------#

In [0]:
#creating the tokenizer for the entire training data
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(analyzer='word',min_df=1,lowercase=True,stop_words=stopwords,smooth_idf=True,sublinear_tf=True)
X=vectorizer.fit_transform(data['essay'])
tf_idf = pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names())


In [0]:
len1=X.toarray().shape[0]

In [0]:
#Initialising the tokenizer
tokenizer =vectorizer.build_tokenizer()

In [0]:
def get_essay_tfidf(essay):
    sentences = tokenizer(essay)
    feature_vector=np.zeros((len1,1))
    count=0
    for w in sentences:
        if not w.lower() in stopwords and w in vectorizer.vocabulary_: 
            feature_vector=feature_vector+tf_idf[w.lower()].values.reshape(-1,1)
            count=count+1
    return feature_vector/count

In [0]:
def create_training_data(essays):
    train=[]
    for essay in essays:
        essay_vector = get_essay_tfidf(essay).tolist()
        train.append(essay_vector)
    return train

In [95]:
#Generating the word vectors using TF-IDF method
train_tfidf=create_training_data(data['essay'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':


In [0]:
#---------------------------Pre trained Word2Vec model ---------------------------------------#

In [0]:
#Getting the word vectors from pretrained word embeddings
def get_pretrained_word2vec_essay(wv,essay,dimension):
    words = essay_to_words(essay)
    feature_vector = np.zeros((1,dimension))
    count=0
    for word in words:
        if word.lower() in wv:
            feature_vector=np.add(feature_vector,wv[word.lower()])
            count=count+1
    return feature_vector/count

In [0]:
def create_training_data_pretrained(wv,essays):
    essay = []
    for e in essays:
        essay.append(vector.tolist())
    return np.array(essay)

In [0]:
#Generating the pretrained word vectors
train_pretrained=create_training_data_pretrained(wv,data['essay'])

In [0]:
#------------------------------------ Trained word2vec model--------------------------------------------------#


In [0]:
#---------------------- Training the word2vec model provided by gensim---------------------------#
def train_model(corpus,window,workers,min_count,features,sample):
    model = Word2Vec(corpus,size=features,min_count=min_count,workers=workers,window=window,sample=sample)
    return model

In [0]:
#------------------------Training the model-----------------#
model = train_model(training_corpus,10,4,1,300,1e-3)

In [0]:
#-------------Generating word vectors----------------------------------#
train_trained=create_training_data_pretrained(model.wv,data['essay'])

In [16]:
#------------------------------------Preprocessing words for Embedding layer-----------------#
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
def convert_tokens_to_integers(essays,m=0):
  corpora,maxlen=create_essay_to_word_index(essays)
  if(m!=0):
    maxlen=m
  padded = pad_sequences(corpora, maxlen=maxlen,padding='post',truncating='post')
  return np.array(padded),maxlen

Using TensorFlow backend.


In [0]:
def create_embedding_matrix(wv):
  embedding_matrix=np.zeros((len(vectorizer.vocabulary_),300))
  for word in vectorizer.vocabulary_:
    if(not check_tagged_label(word.upper())):
      if word.lower() in wv: 
        embedding_matrix[vectorizer.vocabulary_[word]]=wv[word]
  return embedding_matrix

In [0]:
#---------creating the embedding matrix using the trained word2vec model for Embedding Layer----------------------#
a=create_embedding_matrix(model.wv)

In [0]:
test=convert_tokens_to_integers(data['essay'])

In [0]:
#------------------------------model-----------------------------------------#

In [0]:
import tensorflow
from tensorflow import keras
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Flatten, GRU, SimpleRNN,Conv1D,MaxPooling1D
from keras.models import Sequential, load_model, model_from_config

In [0]:
def get_model(hidden_layer_units1=300,hidden_layer_units2=64,use_embeddings=False,use_lstm=False,use_GRU=False,use_RNN=False,use_conv=False,use_bidirection=False,input_shape=[],maxlen=512):
  model = Sequential()
  vocabulary_size=len(vectorizer.vocabulary_.keys())
  embedding_dimension=300
  if(use_embeddings):
    model.add(Embedding(vocabulary_size, embedding_dimension,weights=[a], input_length=maxlen, mask_zero=False))
  if use_conv:
    model.add(Conv1D(filters=50, kernel_size=5, padding='same'))
    model.add(MaxPooling1D(2))
    model.add(Flatten())
  if use_lstm:
    if use_bidirection:
      model.add(Bidirectional(LSTM(hidden_layer_units1, dropout=0.4, recurrent_dropout=0.3, input_shape=input_shape, return_sequences=True)))
      model.add(Bidirectional(LSTM(hidden_layer_units2, recurrent_dropout=0.3)))
    else:
      model.add(LSTM(hidden_layer_units1, dropout=0.4, recurrent_dropout=0.4, input_shape=input_shape, return_sequences=True))
      model.add(LSTM(hidden_layer_units2, recurrent_dropout=0.4))
  if use_RNN:
    if use_bidirection:
      model.add(Bidirectional(SimpleRNN(300, input_shape=input_shape, return_sequences=True)))
      model.add(Bidirectional(SimpleRNN(64)))
    else:
      model.add(SimpleRNN(300,input_shape=input_shape, return_sequences=True))
      model.add(SimpleRNN(64))
  if use_GRU:
    if use_bidirection:
      model.add(Bidirectional(GRU(300, input_shape=input_shape, return_sequences=True)))
      model.add(Bidirectional(GRU(64)))
    else:
      model.add(GRU(300, dropout=0.4, recurrent_dropout=0.4, input_shape=input_shape, return_sequences=True))
      model.add(GRU(64,recurrent_dropout=0.4))
  model.add(Dropout(0.3))
  model.add(Dense(1, activation='relu'))
  model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
  return model

In [0]:
model=get_model(use_conv=False,use_GRU=True,use_bidirection=True,input_shape=train_trained.shape)

In [0]:
#Trainng the neural net model using the trained word2vec model to get a training error estimate-------------------------#
model.fit(train_trained,y,batch_size=100,epochs=10)

Epoch 1/10
12976/12976 [==============================] - 5s 388us/step - loss: 38.4803 - mae: 3.7407
Epoch 2/10
12976/12976 [==============================] - 4s 276us/step - loss: 14.3028 - mae: 2.5346
Epoch 3/10
12976/12976 [==============================] - 4s 276us/step - loss: 10.5847 - mae: 2.3069
Epoch 4/10
12976/12976 [==============================] - 4s 277us/step - loss: 9.7027 - mae: 2.2324
Epoch 5/10
12976/12976 [==============================] - 4s 279us/step - loss: 9.1234 - mae: 2.1915
Epoch 6/10
12976/12976 [==============================] - 4s 275us/step - loss: 8.6152 - mae: 2.1543
Epoch 7/10
12976/12976 [==============================] - 4s 282us/step - loss: 8.4949 - mae: 2.1321
Epoch 8/10
12976/12976 [==============================] - 4s 288us/step - loss: 8.2660 - mae: 2.1143
Epoch 9/10
12976/12976 [==============================] - 4s 289us/step - loss: 8.1882 - mae: 2.1180
Epoch 10/10
12976/12976 [==============================] - 4s 285us/step - loss: 7.9282 

In [0]:
#--------------------------------------- Hyper parameter tuning -------------------------------------------#
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
epochs=[50]
hidden1=[300,400,500]
hidden2=[32,64,128]
param_grid=dict(epochs=epochs,hidden_layer_units1=hidden1,hidden_layer_units2=hidden2)
m = KerasRegressor(build_fn=get_model,use_lstm=True,use_bidirection=True)
grid = GridSearchCV(estimator=m,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(train_trained,y)
grid_result.best_params_

In [0]:
#--------------------------------- Training ---------------------------#
from sklearn.model_selection import KFold
from sklearn.metrics import cohen_kappa_score
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
cv = KFold(n_splits=5,shuffle=True)
kappa_score_pretrain=[]
kappa_score_train=[]
kappa_score_embedding=[]
check=0
for train,test in cv.split(data):
  if check==1:
    break
  data_test,data_train,y_test,y_train = data.iloc[test],data.iloc[train],y.iloc[test],y.iloc[train]
  train = data_train['essay']
  test = data_test['essay']
#--------------- Pretrained vectors------------------#
  train_pretrained=create_training_data_pretrained(wv,train)
  test_pretrained=create_training_data_pretrained(wv,test)
  print(train_pretrained.shape)
#-------------------------- Trained vectors---------------#
  vectorizer = TfidfVectorizer(analyzer='word',min_df=1,lowercase=True,stop_words=stopwords,smooth_idf=True,sublinear_tf=True)
  X=vectorizer.fit_transform(train)
  vocab_size = len(vectorizer.vocabulary_.keys())
  vectorizer.vocabulary_['OOV']=vocab_size
  tokenizer =vectorizer.build_tokenizer()
  training_corpus=create_sentence_vocab(train)
  word2vec = train_model(training_corpus,10,4,1,300,1e-3)
  train_trained = create_training_data_pretrained(word2vec.wv,train)
  test_trained = create_training_data_pretrained(word2vec.wv,test)
#----------------------------Embedding vectors-------------------#
  a=create_embedding_matrix(word2vec.wv)
  train_embedding,maxlen=convert_tokens_to_integers(train)
  test_embedding=convert_tokens_to_integers(test,m=maxlen)
#------------------------------- Define models----------------------------#
  model1 = get_model(use_lstm=True,input_shape=[1,300])
  model2 = get_model(use_GRU=True,use_bidirection=True,input_shape=[1,300])
  model3 = get_model(use_lstm=True,use_bidirection=True,input_shape=[1,300])
  model4 = get_model(use_RNN=True,use_bidirection=True,input_shape=[1,300])
  model5 = get_model(use_embeddings=True,use_lstm=True,use_bidirection=True)
#--------------------------------Pretrained training and testing----------#
  model1.fit(train_pretrained,y_train,batch_size=64,epochs=50)
  ypred_1 = model1.predict(test_pretrained)
  ypred_1 = np.around(ypred_1)
  result1 = cohen_kappa_score(y_test,ypred_1,weights='quadratic')
  kappa_score_pretrain.append(result1)
  model2.fit(train_pretrained,y_train.values.reshape(-1,1),batch_size=64,epochs=50)
  ypred_2 = model2.predict(test_pretrained)
  ypred_2 = np.around(ypred_2)
  result2 = cohen_kappa_score(y_test,ypred_2,weights='quadratic')
  kappa_score_pretrain.append(result2)
  model3.fit(train_pretrained,y_train.values.reshape(-1,1),batch_size=64,epochs=50)
  ypred_3 = model3.predict(test_pretrained)
  ypred_3 = np.around(ypred_3)
  result3 = cohen_kappa_score(y_test,ypred_3,weights='quadratic')
  kappa_score_pretrain.append(result3)
  model4.fit(train_pretrained,y_train.values.reshape(-1,1),batch_size=64,epochs=50)
  ypred_4 = model4.predict(test_pretrained)
  ypred_4 = np.around(ypred_4)
  result4 = cohen_kappa_score(y_test,ypred_4,weights='quadratic')
  kappa_score_pretrain.append(result4)
#---------------------------------Trained vectors--------------------------#
  model3.fit(train_trained,y_train.values.reshape(-1,1),batch_size=64,epochs=50)
  ypred_21 = model3.predict(test_trained)
  ypred_21 = np.around(ypred_21)
  result21 = cohen_kappa_score(y_test,ypred_21,weights='quadratic')
  kappa_score_train.append(result21)
#-----------------------------------Embedding vectors-------------------------------#
  model5.fit(train_embedding,y_train.values.reshape(-1,1),batch_size=64,epochs=50)
  ypred_31 = model5.predict(test_embedding)
  ypred_31 = np.around(ypred_31)
  result31 = cohen_kappa_score(y_test,ypred_31,weights='quadratic')
  kappa_score_embedding.append(result31)
pickle.dump(kappa_score_pretrain,open("file1.pickle","rb"))
pickle.dump(kappa_score_train,open("file2.pickle","wb"))
pickle.dump(kappa_score_embedding,open("file3.pickle","wb"))


In [66]:
print("Accuracy score for trained vectors model 3 "+ str(np.sum(kappa_score_train)/5))

Accuracy score for trained vectors model 3 0.9631036890997183


In [0]:
s1,s2,s3,s4=0,0,0,0
for i in range(0,5):
  s1+=pt[4*i]
  s2+=pt[4*i+1]
  s3+=pt[4*i+2]
  s4+=pt[4*i+3]

In [60]:
print("Accuracy score for pretrained vectors model 1 " + str(s1/5))

Accuracy score for pretrained vectors model 1 0.9680721549776587


In [61]:
print("Accuracy score for pretrained vectors model 2 " + str(s2/5))

Accuracy score for pretrained vectors model 2 0.9640297094956518


In [62]:
print("Accuracy score for pretrained vectors model 3 " + str(s3/5))

Accuracy score for pretrained vectors model 3 0.9683148498381282


In [63]:
print("Accuracy score for pretrained vectors model 4 " + str(s4/5))

Accuracy score for pretrained vectors model 4 0.9536714377736928


In [68]:
print("accuracy score for self trained word embeddings = "+str(kappa_score_embedding[0]))

accuracy score for self trained word embeddings = 0.9726917237214495


In [0]:
#---------------------------- RUnning our model on the test data provided by Kaggle--------------------------------------------------#
test = pd.read_csv('test_set.tsv', encoding = "ISO-8859-1",sep='\t')

In [0]:
#------------------------------ Converting test data into word vectors for our best performing model--------------------------------#
test_set =convert_tokens_to_integers(test['essay'],m=maxlen) 

In [0]:
ytest=model5.predict(test_set[0])

In [0]:
test['domain_score_graded'] = np.around(ytest)

In [56]:
test.head(10)

,essay_id,essay_set,essay,domain1_predictionid,domain2_predictionid,domain_score_graded
0,2383,1,I believe that computers have a positive effec...,2383,NaN,8.0
1,2384,1,"Dear @CAPS1, I know some problems have came up...",2384,NaN,9.0
2,2385,1,"Dear to whom it @MONTH1 concern, Computers are...",2385,NaN,6.0
3,2386,1,"Dear @CAPS1 @CAPS2, @CAPS3 has come to my atte...",2386,NaN,9.0
4,2387,1,"Dear Local newspaper, I think that people have...",2387,NaN,9.0
5,2388,1,"Dear @CAPS1, @CAPS2 is in my firm belief that ...",2388,NaN,10.0
6,2389,1,"Dear local newspaper, @CAPS1 your computer hav...",2389,NaN,8.0
7,2390,1,"Dear @CAPS1, I agreed with the people that fee...",2390,NaN,7.0
8,2391,1,"Dear @CAPS1 @CAPS2, I do not believe that comp...",2391,NaN,9.0
9,2392,1,Imagine that you walk into a friends home and ...,2392,NaN,9.0


In [0]:
test.to_csv('test_results.csv')